## Libraries

In [1]:
import sys 
sys.path.append(r"C:\Pro\Stages\A4 - DVRC\Work\Supervised-Learning-using-Unsupervised-Learning-Metrics-in-the-absence-of-Annotated-Data\myLibraries")
sys.path.append(r"C:\Pro\Stages\A4 - DVRC\Work\Supervised-Learning-using-Unsupervised-Learning-Metrics-in-the-absence-of-Annotated-Data")
sys.path.append(r"C:\Pro\Stages\A4 - DVRC\Work\Supervised-Learning-using-Unsupervised-Learning-Metrics-in-the-absence-of-Annotated-Data\MENLI")

In [2]:
from custom_score.score import BERTScoreStaticSampleTest, BERTScoreDynamicSampleTest, BARTScoreDynamicSampleTest, MENLIScoreSampleTest
from custom_score.utils import serialized_to_model
import torch
import tensorflow_datasets as tfds
import pandas as pd

### Checkup and linkage

In [3]:
torch.cuda.is_available()

True

## Datasets

### Billsum

In [4]:
billsumTest_url='https://drive.google.com/file/d/1Wd0M3qepNF6B4YwFYrpo7CaSERpudAG_/view?usp=share_link'
billsumTest_url='https://drive.google.com/uc?id=' + billsumTest_url.split('/')[-2]
billsum_test = pd.read_json(billsumTest_url, lines=True)
billsum_test = billsum_test.loc[:, ["text", "summary"]]
billsum_test.head(5)

,text,summary
0,SECTION 1. SHORT TITLE.\n\n This Act may be...,National Science Education Tax Incentive for B...
1,SECTION 1. SHORT TITLE.\n\n This Act may be...,Small Business Expansion and Hiring Act of 201...
2,SECTION 1. RELEASE OF DOCUMENTS CAPTURED IN IR...,Requires the Director of National Intelligence...
3,SECTION 1. SHORT TITLE.\n\n This Act may be...,National Cancer Act of 2003 - Amends the Publi...
4,SECTION 1. SHORT TITLE.\n\n This Act may be...,Military Call-up Relief Act - Amends the Inter...


### Gigaword

In [5]:
gigaword_builder = tfds.builder("Gigaword")
gigaword_builder.download_and_prepare()
gigaword = gigaword_builder.as_dataset()

In [6]:
gigaword = tfds.as_dataframe(gigaword["test"])
gigaword = gigaword.rename(columns={"document":"text", "summary":"summary"})
gigaword['summary'] = gigaword['summary'].str.decode("utf-8").str.strip().str.replace("#", "")
gigaword['text'] = gigaword['text'].str.decode("utf-8").str.strip().str.replace("#", "")
gigaword.head(5)

,text,summary
0,the german government and red cross have decid...,"germany gives , dollars in aid for iran quake ..."
1,teenager michaela krajicek claimed her first g...,krajicek defeats safina in dutch final UNK pic...
2,after spending years in prison for a killing ...,"$ , for years wrongly held in prison"
3,the first probable human case of mad cow disea...,st case of probable mad cow disease listed pos...
4,at least people were killed as typhoon sepat ...,"storms lash chinese coast killing ; , evacuat..."


### Multinews

In [7]:
multinews_builder = tfds.builder("MultiNews")
multinews_builder.download_and_prepare()
multinews = multinews_builder.as_dataset()

In [8]:
multinews = tfds.as_dataframe(multinews["test"])
multinews = multinews.rename(columns={"document":"text", "summary":"summary"})
multinews['summary'] = multinews['summary'].str.decode("utf-8").str.replace("\xe2\x80\x93", "").str[2:].str.strip()
multinews['text'] = multinews['text'].str.decode("utf-8").str.strip()
multinews.head(5)

,text,summary
0,"Think back, everyone -- can you remember when ...",No matter how much you like Jason Bateman and ...
1,'Alien: Covenant': Film Review \n \n Michael ...,A spaceship arrives on a distant planet that l...
2,By REUTERS \n \n Photo: Reuters \n \n Charli...,France is on lockdown today after a satirical ...
3,These crawls are part of an effort to archive ...,Just when you thought the Republican president...
4,The estranged wife of acclaimed concert pianis...,"Sofya Tsygankova, the estranged wife of famed ..."


### PubMed

In [9]:
pm_path = r"C:\Pro\Stages\A4 - DVRC\Work\Datasets\pubmed\test.json"
pubmed_test = pd.read_json(pm_path, lines=True)
pm = pubmed_test[["article_text", "abstract_text"]]
cleaner = lambda x: ". ".join(x).replace("<S>", "").strip()

pm.loc[:,"abstract_text"] = pm["abstract_text"].replace(regex=r"\[[^\]]*\]", value="")
pm.loc[:,"article_text"] = pm["article_text"].replace(regex=r"\[[^\]]*\]", value="")
pm.loc[:,"abstract_text"] = pm["abstract_text"].map(cleaner)
pm.loc[:,"article_text"] = pm["article_text"].map(cleaner)
pm = pm.rename(columns={"article_text":"text", "abstract_text":"summary"})
pubmed = pm.copy()

del pm 
del pubmed_test

pubmed.head()

,text,summary
0,anxiety affects quality of life in those livin...,research on the implications of anxiety in par...
1,small non - coding rnas are transcribed into m...,"small non - coding rnas include sirna , mirna ..."
2,ohss is a serious complication of ovulation in...,objective : to evaluate the efficacy and safet...
3,congenital adrenal hyperplasia ( cah ) refers ...,congenital adrenal hyperplasia is a group of a...
4,type 1 diabetes ( t1d ) results from the destr...,objective(s):pentoxifylline is an immunomodula...


## Benchmark

In [10]:
results = {}
lim = 10

### Billsum

In [11]:
w2v = serialized_to_model(r'C:\Pro\Stages\A4 - DVRC\Work\Models\serialized_w2v.pkl')
bert_scores, bert_runtime = BERTScoreDynamicSampleTest(billsum_test, limit=lim)
word2vec_scores, word2vec_runtime = BERTScoreStaticSampleTest(billsum_test, w2v, lim, withIdf = False)
bart_scores, bart_runtime = BARTScoreDynamicSampleTest(billsum_test, limit=lim)
menli_scores, menli_runtime = MENLIScoreSampleTest(billsum_test, limit=lim)

computing metric scores....
computing nli scores (direction avg + formula e)...
averaging....
combing scores...
computing metric scores....


c:\Pro\Stages\A4 - DVRC\Work\Supervised-Learning-using-Unsupervised-Learning-Metrics-in-the-absence-of-Annotated-Data\.venv\Lib\site-packages\bert_score\scorer.py:178: UserWarning: Overwriting the previous importance weights.
  warnings.warn("Overwriting the previous importance weights.")


computing nli scores (direction avg + formula e)...
averaging....
combing scores...
computing metric scores....
computing nli scores (direction avg + formula e)...
averaging....
combing scores...
computing metric scores....
computing nli scores (direction avg + formula e)...
averaging....
combing scores...
computing metric scores....
computing nli scores (direction avg + formula e)...
averaging....
combing scores...
computing metric scores....
computing nli scores (direction avg + formula e)...
averaging....
combing scores...
computing metric scores....
computing nli scores (direction avg + formula e)...
averaging....
combing scores...
computing metric scores....
computing nli scores (direction avg + formula e)...
averaging....
combing scores...
computing metric scores....
computing nli scores (direction avg + formula e)...
averaging....
combing scores...
computing metric scores....
computing nli scores (direction avg + formula e)...
averaging....
combing scores...


In [12]:
billsum_results = {'BERTScore': [bert_scores ,bert_runtime], 'Custom': [word2vec_scores, word2vec_runtime], 'Bart': [bart_scores, bart_runtime], 'Menli': [menli_scores, menli_runtime]}
results['billsum'] = billsum_results

### Multinews

In [13]:
bert_scores, bert_runtime = BERTScoreDynamicSampleTest(multinews, limit=lim)
word2vec_scores, word2vec_runtime = BERTScoreStaticSampleTest(multinews, w2v, lim, withIdf = False)
bart_scores, bart_runtime = BARTScoreDynamicSampleTest(multinews, limit=lim)
menli_scores, menli_runtime = MENLIScoreSampleTest(billsum_test, limit=lim)

computing metric scores....
computing nli scores (direction avg + formula e)...
averaging....
combing scores...
computing metric scores....
computing nli scores (direction avg + formula e)...
averaging....
combing scores...
computing metric scores....
computing nli scores (direction avg + formula e)...
averaging....
combing scores...
computing metric scores....
computing nli scores (direction avg + formula e)...
averaging....
combing scores...
computing metric scores....
computing nli scores (direction avg + formula e)...
averaging....
combing scores...
computing metric scores....
computing nli scores (direction avg + formula e)...
averaging....
combing scores...
computing metric scores....
computing nli scores (direction avg + formula e)...
averaging....
combing scores...
computing metric scores....
computing nli scores (direction avg + formula e)...
averaging....
combing scores...
computing metric scores....
computing nli scores (direction avg + formula e)...
averaging....
combing sc

In [14]:
multinews_results = {'BERTScore': [bert_scores ,bert_runtime], 'Custom': [word2vec_scores, word2vec_runtime], 'Bart': [bart_scores, bart_runtime], 'Menli': [menli_scores, menli_runtime]}
results['multinews'] = multinews_results

### Gigaword

In [15]:
bert_scores, bert_runtime = BERTScoreDynamicSampleTest(gigaword, limit=lim)
word2vec_scores, word2vec_runtime = BERTScoreStaticSampleTest(gigaword, w2v, lim, withIdf = False)
bart_scores, bart_runtime = BARTScoreDynamicSampleTest(gigaword, limit=lim)
menli_scores, menli_runtime = MENLIScoreSampleTest(billsum_test, limit=lim)

computing metric scores....
computing nli scores (direction avg + formula e)...
averaging....
combing scores...
computing metric scores....
computing nli scores (direction avg + formula e)...
averaging....
combing scores...
computing metric scores....
computing nli scores (direction avg + formula e)...
averaging....
combing scores...
computing metric scores....
computing nli scores (direction avg + formula e)...
averaging....
combing scores...
computing metric scores....
computing nli scores (direction avg + formula e)...
averaging....
combing scores...
computing metric scores....
computing nli scores (direction avg + formula e)...
averaging....
combing scores...
computing metric scores....
computing nli scores (direction avg + formula e)...
averaging....
combing scores...
computing metric scores....
computing nli scores (direction avg + formula e)...
averaging....
combing scores...
computing metric scores....
computing nli scores (direction avg + formula e)...
averaging....
combing sc

In [16]:
gigaword_results = {'BERTScore': [bert_scores ,bert_runtime], 'Custom': [word2vec_scores, word2vec_runtime], 'Bart': [bart_scores, bart_runtime], 'Menli': [menli_scores, menli_runtime]}
results['gigaword'] = gigaword_results

### PubMed

In [17]:
bert_scores, bert_runtime = BERTScoreDynamicSampleTest(pubmed, limit=lim)
word2vec_scores, word2vec_runtime = BERTScoreStaticSampleTest(pubmed, w2v, lim, withIdf = False)
bart_scores, bart_runtime = BARTScoreDynamicSampleTest(pubmed, limit=lim)
menli_scores, menli_runtime = MENLIScoreSampleTest(billsum_test, limit=lim)

computing metric scores....
computing nli scores (direction avg + formula e)...
averaging....
combing scores...
computing metric scores....
computing nli scores (direction avg + formula e)...
averaging....
combing scores...
computing metric scores....
computing nli scores (direction avg + formula e)...
averaging....
combing scores...
computing metric scores....
computing nli scores (direction avg + formula e)...
averaging....
combing scores...
computing metric scores....
computing nli scores (direction avg + formula e)...
averaging....
combing scores...
computing metric scores....
computing nli scores (direction avg + formula e)...
averaging....
combing scores...
computing metric scores....
computing nli scores (direction avg + formula e)...
averaging....
combing scores...
computing metric scores....
computing nli scores (direction avg + formula e)...
averaging....
combing scores...
computing metric scores....
computing nli scores (direction avg + formula e)...
averaging....
combing sc

In [18]:
pubmed_results = {'BERTScore': [bert_scores ,bert_runtime], 'Custom': [word2vec_scores, word2vec_runtime], 'Bart': [bart_scores, bart_runtime], 'Menli': [menli_scores, menli_runtime]}
results['pubmed'] = pubmed_results

## Evalutation

### Runtime Table

In [19]:
billsum_runtimes = [results["billsum"]["BERTScore"][1], results["billsum"]["Custom"][1], results["billsum"]["Bart"][1], results["billsum"]["Menli"]]
multinews_runtimes = [results["multinews"]["BERTScore"][1], results["multinews"]["Custom"][1], results["multinews"]["Bart"][1], results["multinews"]["Menli"]]
gigaword_runtimes = [results["gigaword"]["BERTScore"][1], results["gigaword"]["Custom"][1], results["gigaword"]["Bart"][1], results["gigaword"]["Menli"]]
pubmed_runtimes = [results["pubmed"]["BERTScore"][1], results["pubmed"]["Custom"][1], results["pubmed"]["Bart"][1], results["pubmed"]["Menli"]]

runtimeDf = pd.DataFrame({"Billsum": billsum_runtimes, 
                          "Multinews": multinews_runtimes,
                          "Gigaword": gigaword_runtimes,
                          "PubMed": pubmed_runtimes}, 
                         index=["Roberta-24-layers", "Word2Vec", "BART-large-CNN", "Menli"])
runtimeDf

,Billsum,Multinews,Gigaword,PubMed
Roberta-24-layers,50.666811,45.809743,33.539054,35.836471
Word2Vec,27.191216,42.139481,0.085041,83.065523
BART-large-CNN,2.011301,1.209128,0.478114,1.369402
Menli,"[[0.19761285781860352, 0.18756558895111083, 0....","[[0.19761285781860352, 0.18756558895111083, 0....","[[0.19761285781860352, 0.18756558895111083, 0....","[[0.19761285781860352, 0.18756558895111083, 0...."


### Quality evaluation

#### Billsum

In [20]:
billsumQualityDf = pd.concat((pd.DataFrame(results["billsum"]["Custom"][0], columns=["W2V_P", "W2V_R", "W2V_F"]),
                              pd.DataFrame(results["billsum"]["BERTScore"][0], columns=["Bert_P", "Bert_R", "Bert_F"]),
                              pd.DataFrame(results["billsum"]["Bart"][0], columns=["Bart"]),
                              pd.DataFrame(results["billsum"]["Menli"][0], columns=["Menli"])),
                              axis=1)
billsumQualityDf.head(5)

,W2V_P,W2V_R,W2V_F,Bert_P,Bert_R,Bert_F,Bart,Menli
0,0.542677,0.904048,0.678230,0.874369,0.704338,0.780197,0.392246,0.197613
1,0.769379,0.953081,0.851434,0.846002,0.728316,0.782760,0.393373,0.187566
2,0.718532,0.909344,0.802755,0.873844,0.702725,0.778998,0.338536,0.171082
3,0.618058,0.895988,0.731513,0.810689,0.703450,0.753272,0.271333,0.100825
4,0.480467,0.883006,0.622316,0.901381,0.665327,0.765571,0.481292,0.258599


#### Multinews

In [21]:
multinewsQualityDf = pd.concat((pd.DataFrame(results["multinews"]["Custom"][0], columns=["W2V_P", "W2V_R", "W2V_F"]),
                              pd.DataFrame(results["multinews"]["BERTScore"][0], columns=["Bert_P", "Bert_R", "Bert_F"]),
                              pd.DataFrame(results["multinews"]["Bart"][0], columns=["Bart"]),
                              pd.DataFrame(results["multinews"]["Menli"][0], columns=["Menli"])),
                              axis=1)
multinewsQualityDf.head(5)

,W2V_P,W2V_R,W2V_F,Bert_P,Bert_R,Bert_F,Bart,Menli
0,0.648935,0.854094,0.737513,0.810646,0.794206,0.802342,0.259141,0.197613
1,0.656565,0.880320,0.752154,0.841052,0.826047,0.833482,0.329445,0.187566
2,0.651669,0.863386,0.742735,0.863537,0.827867,0.845326,0.340231,0.171082
3,0.617941,0.856395,0.717885,0.862562,0.805295,0.832945,0.516795,0.100825
4,0.687182,0.878509,0.771155,0.914194,0.850664,0.881285,0.547971,0.258599


#### Gigaword

In [22]:
gigawordQualityDf = pd.concat((pd.DataFrame(results["gigaword"]["Custom"][0], columns=["W2V_P", "W2V_R", "W2V_F"]),
                              pd.DataFrame(results["gigaword"]["BERTScore"][0], columns=["Bert_P", "Bert_R", "Bert_F"]),
                              pd.DataFrame(results["gigaword"]["Bart"][0], columns=["Bart"]),
                              pd.DataFrame(results["gigaword"]["Menli"][0], columns=["Menli"])),
                              axis=1)
gigawordQualityDf.head(5)

,W2V_P,W2V_R,W2V_F,Bert_P,Bert_R,Bert_F,Bart
0,0.520848,0.858084,0.648229,0.931568,0.861043,0.894918,0.270875
1,0.426644,0.616523,0.504302,0.878367,0.817973,0.847095,0.373961
2,0.397880,0.683416,0.502947,0.882140,0.839132,0.860099,0.072829
3,0.622144,0.939688,0.748635,0.947691,0.876814,0.910876,0.444639
4,0.445092,0.737083,0.555027,0.878134,0.835666,0.856374,0.110215


#### PubMed

In [23]:
pubmedQualityDf = pd.concat((pd.DataFrame(results["pubmed"]["Custom"][0], columns=["W2V_P", "W2V_R", "W2V_F"]),
                              pd.DataFrame(results["pubmed"]["BERTScore"][0], columns=["Bert_P", "Bert_R", "Bert_F"]),
                              pd.DataFrame(results["pubmed"]["Bart"][0], columns=["Bart"]),
                              pd.DataFrame(results["pubmed"]["Menli"][0], columns=["Menli"])),
                              axis=1)
pubmedQualityDf.head(5)

,W2V_P,W2V_R,W2V_F,Bert_P,Bert_R,Bert_F,Bart,Menli
0,0.656127,0.952100,0.776879,0.854162,0.842896,0.848492,0.287621,0.197613
1,0.601416,0.954034,0.737756,0.861836,0.812966,0.836688,0.385551,0.187566
2,0.628735,0.966463,0.761847,0.806066,0.786019,0.795916,0.236114,0.171082
3,0.662453,0.992789,0.794659,0.855671,0.825281,0.840201,0.439659,0.100825
4,0.718312,0.963990,0.823212,0.828977,0.824318,0.826641,0.401576,0.258599


#### Summary

In [24]:
meanGigawordQualityDf = gigawordQualityDf.mean(axis=0)
meanMultinewsQualityDf = multinewsQualityDf.mean(axis=0)
meanBillsumQualityDf = billsumQualityDf.mean(axis=0)
meanPubmedQualityDf = pubmedQualityDf.mean(axis=0)

summaryDf = pd.DataFrame({"Gigaword":meanGigawordQualityDf,
                          "Billsum": meanBillsumQualityDf,
                          "Multinews": meanMultinewsQualityDf,
                          "PubMed": meanPubmedQualityDf})
summaryDf

,Gigaword,Billsum,Multinews,PubMed
Bart,0.279085,0.374697,0.387969,0.309953
Bert_F,0.871343,0.765928,0.838067,0.817707
Bert_P,0.905045,0.852451,0.859192,0.829127
Bert_R,0.840408,0.696554,0.818171,0.807103
Menli,NaN,0.133496,0.133496,0.133496
W2V_F,0.551814,0.741452,0.747495,0.780022
W2V_P,0.438547,0.630637,0.654471,0.660659
W2V_R,0.757547,0.907242,0.872364,0.965610


## Shuffled Data

### Shuffling datasets

In [25]:
shuffledGigaword = gigaword.copy()
shuffledGigaword["summary"] = shuffledGigaword["summary"].sample(frac=1).values

shuffledMultinews = multinews.copy()
shuffledMultinews["summary"] = shuffledMultinews["summary"].sample(frac=1).values

shuffledBillsum = billsum_test.copy()
shuffledBillsum["summary"] = shuffledBillsum["summary"].sample(frac=1).values

shuffledPubmed= pubmed.copy()
shuffledPubmed["summary"] = shuffledPubmed["summary"].sample(frac=1).values

In [26]:
shuffledMultinews.head(3)

,text,summary
0,"Think back, everyone -- can you remember when ...",The number of pregnant women in the US and its...
1,'Alien: Covenant': Film Review \n \n Michael ...,De La Soul has found a pretty cool way to cele...
2,By REUTERS \n \n Photo: Reuters \n \n Charli...,Caitlyn Jenner has a new memoir dropping April...


In [27]:
shuffledBillsum.head(3)

,text,summary
0,SECTION 1. SHORT TITLE.\n\n This Act may be...,Supporting Emergency Responders Volunteer Effo...
1,SECTION 1. SHORT TITLE.\n\n This Act may be...,Simple Universal Healthcare Act of 2009 - Requ...
2,SECTION 1. RELEASE OF DOCUMENTS CAPTURED IN IR...,Television Viewer Privacy Act of 2003 - Prohib...


In [28]:
shuffledGigaword.head(3)

,text,summary
0,the german government and red cross have decid...,us deports to bosnia two serbs wanted for sreb...
1,teenager michaela krajicek claimed her first g...,mir 's problems at a glance
2,after spending years in prison for a killing ...,police turn UNK in insurgency-hit indian state...


In [29]:
shuffledPubmed.head(3)

,text,summary
0,anxiety affects quality of life in those livin...,clinical usage of several classes of antibioti...
1,small non - coding rnas are transcribed into m...,goal : the goal of the research was to determi...
2,ohss is a serious complication of ovulation in...,introduction : a coracoid fracture is a rare o...


### Evaluating the forged datasets

In [30]:
results = {}
lim = 10

w2v = serialized_to_model(r'C:\Pro\Stages\A4 - DVRC\Work\Models\serialized_w2v.pkl')
bert_scores, bert_runtime = BERTScoreDynamicSampleTest(billsum_test, limit=lim)
word2vec_scores, word2vec_runtime = BERTScoreStaticSampleTest(billsum_test, w2v, lim, withIdf = False)
bart_scores, bart_runtime = BARTScoreDynamicSampleTest(billsum_test, limit=lim)
menli_scores, menli_runtime = MENLIScoreSampleTest(billsum_test, limit=lim)
billsum_results = {'BERTScore': [bert_scores ,bert_runtime], 'Custom': [word2vec_scores, word2vec_runtime], 'Bart': [bart_scores, bart_runtime], 'Menli': [menli_scores, menli_runtime]}
results['billsum'] = billsum_results

bert_scores, bert_runtime = BERTScoreDynamicSampleTest(multinews, limit=lim)
word2vec_scores, word2vec_runtime = BERTScoreStaticSampleTest(multinews, w2v, lim, withIdf = False)
bart_scores, bart_runtime = BARTScoreDynamicSampleTest(multinews, limit=lim)
menli_scores, menli_runtime = MENLIScoreSampleTest(billsum_test, limit=lim)
multinews_results = {'BERTScore': [bert_scores ,bert_runtime], 'Custom': [word2vec_scores, word2vec_runtime], 'Bart': [bart_scores, bart_runtime], 'Menli': [menli_scores, menli_runtime]}
results['multinews'] = multinews_results

bert_scores, bert_runtime = BERTScoreDynamicSampleTest(gigaword, limit=lim)
word2vec_scores, word2vec_runtime = BERTScoreStaticSampleTest(gigaword, w2v, lim, withIdf = False)
bart_scores, bart_runtime = BARTScoreDynamicSampleTest(gigaword, limit=lim)
menli_scores, menli_runtime = MENLIScoreSampleTest(billsum_test, limit=lim)
gigaword_results = {'BERTScore': [bert_scores ,bert_runtime], 'Custom': [word2vec_scores, word2vec_runtime], 'Bart': [bart_scores, bart_runtime], 'Menli': [menli_scores, menli_runtime]}
results['gigaword'] = gigaword_results

bert_scores, bert_runtime = BERTScoreDynamicSampleTest(pubmed, limit=lim)
word2vec_scores, word2vec_runtime = BERTScoreStaticSampleTest(pubmed, w2v, lim, withIdf = False)
bart_scores, bart_runtime = BARTScoreDynamicSampleTest(pubmed, limit=lim)
menli_scores, menli_runtime = MENLIScoreSampleTest(billsum_test, limit=lim)
pubmed_results = {'BERTScore': [bert_scores ,bert_runtime], 'Custom': [word2vec_scores, word2vec_runtime], 'Bart': [bart_scores, bart_runtime], 'Menli': [menli_scores, menli_runtime]}
results['pubmed'] = pubmed_results

computing metric scores....
computing nli scores (direction avg + formula e)...
averaging....
combing scores...
computing metric scores....


c:\Pro\Stages\A4 - DVRC\Work\Supervised-Learning-using-Unsupervised-Learning-Metrics-in-the-absence-of-Annotated-Data\.venv\Lib\site-packages\bert_score\scorer.py:178: UserWarning: Overwriting the previous importance weights.
  warnings.warn("Overwriting the previous importance weights.")


computing nli scores (direction avg + formula e)...
averaging....
combing scores...
computing metric scores....
computing nli scores (direction avg + formula e)...
averaging....
combing scores...
computing metric scores....
computing nli scores (direction avg + formula e)...
averaging....
combing scores...
computing metric scores....
computing nli scores (direction avg + formula e)...
averaging....
combing scores...
computing metric scores....
computing nli scores (direction avg + formula e)...
averaging....
combing scores...
computing metric scores....
computing nli scores (direction avg + formula e)...
averaging....
combing scores...
computing metric scores....
computing nli scores (direction avg + formula e)...
averaging....
combing scores...
computing metric scores....
computing nli scores (direction avg + formula e)...
averaging....
combing scores...
computing metric scores....
computing nli scores (direction avg + formula e)...
averaging....
combing scores...
computing metric sco

In [31]:
billsumQualityDf = pd.concat((pd.DataFrame(results["billsum"]["Custom"][0], columns=["W2V_P", "W2V_R", "W2V_F"]),
                              pd.DataFrame(results["billsum"]["BERTScore"][0], columns=["Bert_P", "Bert_R", "Bert_F"]),
                              pd.DataFrame(results["billsum"]["Bart"][0], columns=["Bart"]),
                              pd.DataFrame(results["billsum"]["Menli"][0], columns=["Menli"])),
                              axis=1)

multinewsQualityDf = pd.concat((pd.DataFrame(results["multinews"]["Custom"][0], columns=["W2V_P", "W2V_R", "W2V_F"]),
                              pd.DataFrame(results["multinews"]["BERTScore"][0], columns=["Bert_P", "Bert_R", "Bert_F"]),
                              pd.DataFrame(results["multinews"]["Bart"][0], columns=["Bart"]),
                              pd.DataFrame(results["multinews"]["Menli"][0], columns=["Menli"])),
                              axis=1)

gigawordQualityDf = pd.concat((pd.DataFrame(results["gigaword"]["Custom"][0], columns=["W2V_P", "W2V_R", "W2V_F"]),
                              pd.DataFrame(results["gigaword"]["BERTScore"][0], columns=["Bert_P", "Bert_R", "Bert_F"]),
                              pd.DataFrame(results["gigaword"]["Bart"][0], columns=["Bart"]),
                              pd.DataFrame(results["gigaword"]["Menli"][0], columns=["Menli"])),
                              axis=1)

pubmedQualityDf = pd.concat((pd.DataFrame(results["pubmed"]["Custom"][0], columns=["W2V_P", "W2V_R", "W2V_F"]),
                              pd.DataFrame(results["pubmed"]["BERTScore"][0], columns=["Bert_P", "Bert_R", "Bert_F"]),
                              pd.DataFrame(results["pubmed"]["Bart"][0], columns=["Bart"]),
                              pd.DataFrame(results["pubmed"]["Menli"][0], columns=["Menli"])),
                              axis=1)
meanGigawordQualityDf = gigawordQualityDf.mean(axis=0)
meanMultinewsQualityDf = multinewsQualityDf.mean(axis=0)
meanBillsumQualityDf = billsumQualityDf.mean(axis=0)
meanPubmedQualityDf = pubmedQualityDf.mean(axis=0)

summaryDf = pd.DataFrame({"Gigaword":meanGigawordQualityDf,
                          "Billsum": meanBillsumQualityDf,
                          "Multinews": meanMultinewsQualityDf,
                          "Pubmed": meanPubmedQualityDf})
summaryDf

,Gigaword,Billsum,Multinews,Pubmed
W2V_P,0.438547,0.630637,0.654471,0.660659
W2V_R,0.757547,0.907242,0.872364,0.965610
W2V_F,0.551814,0.741452,0.747495,0.780022
Bert_P,0.905045,0.852451,0.859192,0.829127
Bert_R,0.840408,0.696554,0.818171,0.807103
Bert_F,0.871343,0.765928,0.838067,0.817707
Bart,0.279085,0.374697,0.387969,0.309953
Menli,0.133496,0.133496,0.133496,0.133496
